<a href="https://colab.research.google.com/github/NanditaaRamakrishnan/CryptoStockNews-SummarizationSentiment/blob/main/StockCryptoNews_SummarizationSentiment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install and Import Dependancies 

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 20.9 MB/s eta 0:00:00


In [ ]:
 !pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.5 MB/s eta 0:00:00


In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from bs4 import BeautifulSoup
import requests

Set Up Summarization Model

In [ ]:
model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

Summarize a Single Article 

In [ ]:
url = "https://economictimes.indiatimes.com/markets/expert-view/better-fund-utilization-helped-to-expand-our-nims-umesh-revankar/articleshow/97547538.cms"
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
paragraphs = soup.find_all('p')

In [ ]:
paragraphs[0].text

'Don’t miss out on ET Prime stories! Get your daily dose of business updates on WhatsApp. click here!'

In [ ]:
text = [paragraph.text for paragraph in paragraphs]
words = ' '.join(text).split(' ')[:400]
ARTICLE = ' '.join(words)

In [ ]:
ARTICLE


'Don’t miss out on ET Prime stories! Get your daily dose of business updates on WhatsApp. click here! The fall in most Adani Group stocks continued on Thursday, wiping out a further ₹1.34 lakh crore off their market capitalisation, totaling ₹8.79 lakh crore or $110 billion of erosion in six trading sessions. Tata Sons has been exploring the possibility of being exempted from a Reserve Bank of India (RBI) notification, issued in October 2021, that included it among the top 10 non-banking financial companies (NBFCs) in terms of asset size, said people with knowledge of the matter. Tata Sons is the holding company of the Tata Group companies.  India’s public sector financial institutions are in a very strong position and their exposure to one private company would not create difficulty for them, finance secretary TV Somanathan told ET in response to a question on developments at the Adani Group. Download The Economic Times News App to get Daily Market Updates & Live Business News. 11 Adva

In [ ]:
input_ids = tokenizer.encode(ARTICLE, return_tensors='pt')
output = model.generate(input_ids, max_length=55, num_beams=5, early_stopping=True)
summary = tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
summary

'Tata Sons exploring possibility of being exempted from RBI notification. Public sector financial institutions in strong position, finance secretary says'

News and Sentiment Pipeline 

In [ ]:
monitored_tickers = ['GME', 'TSLA', 'BTC']

In [ ]:
#Serach for stock news using Google and Yahoo Finance
def search_for_stock_news_urls(ticker):
    search_url = "https://www.google.com/search?q=yahoo+finance+{}&tbm=nws".format(ticker)
    r = requests.get(search_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    atags = soup.find_all('a')
    hrefs = [link['href'] for link in atags]
    return hrefs 

In [ ]:
raw_urls = {ticker:search_for_stock_news_urls(ticker) for ticker in monitored_tickers}
raw_urls

{'GME': ['/?sa=X&ved=0ahUKEwiqrKmZmPn8AhUARzABHWISBdIQOwgC',
  '/search?q=yahoo+finance+GME&tbm=nws&ie=UTF-8&gbv=1&sei=AujcY6qjB4COwbkP4qSUkA0',
  '/search?q=yahoo+finance+GME&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwiqrKmZmPn8AhUARzABHWISBdIQ_AUIBSgA',
  '/search?q=yahoo+finance+GME&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwiqrKmZmPn8AhUARzABHWISBdIQ_AUIBygC',
  '/search?q=yahoo+finance+GME&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwiqrKmZmPn8AhUARzABHWISBdIQ_AUICCgD',
  'https://maps.google.com/maps?q=yahoo+finance+GME&um=1&ie=UTF-8&sa=X&ved=0ahUKEwiqrKmZmPn8AhUARzABHWISBdIQ_AUICSgE',
  '/search?q=yahoo+finance+GME&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwiqrKmZmPn8AhUARzABHWISBdIQ_AUICigF',
  '/search?q=yahoo+finance+GME&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwiqrKmZmPn8AhUARzABHWISBdIQ_AUICygG',
  '/advanced_search',
  '/search?q=yahoo+finance+GME&ie=UTF-8&tbm=nws&source=lnt&tbs=qdr:h&sa=X&ved=0ahUKEwiqrKmZmPn8AhUARzABHWISBdIQpwUIDQ',
  '/search?q=yahoo+finance+GME&ie=U

In [ ]:
raw_urls

{'GME': ['/?sa=X&ved=0ahUKEwiqrKmZmPn8AhUARzABHWISBdIQOwgC',
  '/search?q=yahoo+finance+GME&tbm=nws&ie=UTF-8&gbv=1&sei=AujcY6qjB4COwbkP4qSUkA0',
  '/search?q=yahoo+finance+GME&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwiqrKmZmPn8AhUARzABHWISBdIQ_AUIBSgA',
  '/search?q=yahoo+finance+GME&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwiqrKmZmPn8AhUARzABHWISBdIQ_AUIBygC',
  '/search?q=yahoo+finance+GME&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwiqrKmZmPn8AhUARzABHWISBdIQ_AUICCgD',
  'https://maps.google.com/maps?q=yahoo+finance+GME&um=1&ie=UTF-8&sa=X&ved=0ahUKEwiqrKmZmPn8AhUARzABHWISBdIQ_AUICSgE',
  '/search?q=yahoo+finance+GME&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwiqrKmZmPn8AhUARzABHWISBdIQ_AUICigF',
  '/search?q=yahoo+finance+GME&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwiqrKmZmPn8AhUARzABHWISBdIQ_AUICygG',
  '/advanced_search',
  '/search?q=yahoo+finance+GME&ie=UTF-8&tbm=nws&source=lnt&tbs=qdr:h&sa=X&ved=0ahUKEwiqrKmZmPn8AhUARzABHWISBdIQpwUIDQ',
  '/search?q=yahoo+finance+GME&ie=U

In [ ]:
#Strip out unwanted URLs
import re

In [ ]:
exclude_list = ['maps', 'policies', 'preferences', 'accounts', 'support']

In [ ]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls: 
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [ ]:
cleaned_urls = {ticker:strip_unwanted_urls(raw_urls[ticker], exclude_list) for ticker in monitored_tickers}
cleaned_urls

{'GME': ['https://finance.yahoo.com/news/bed-bath-beyond-timeline-loan-default-bankruptcy-212526437.html',
  'https://finance.yahoo.com/news/meme-stock-mania-5-lessons-220010930.html',
  'https://finance.yahoo.com/news/12-best-one-dollar-stocks-143215032.html',
  'https://ca.finance.yahoo.com/news/bed-bath-beyond-surged-105-185829217.html',
  'https://finance.yahoo.com/news/strength-seen-gamestop-gme-14-144602277.html',
  'https://finance.yahoo.com/news/why-game-stop-stock-is-probably-dead-money-for-a-while-according-to-one-analyst-180851839.html',
  'https://finance.yahoo.com/news/tesla-arkk-meme-stocks-rally-2021-energy-technology-markets-195153458.html',
  'https://www.google.com/search?q%3Dyahoo%2Bfinance%2BGME%26tbm%3Dnws%26pccc%3D1',
  'https://finance.yahoo.com/news/midday-movers-airline-stocks-tesla-135534803.html',
  'https://finance.yahoo.com/news/meme-stocks-i-bought-amc-thinking-it-would-be-the-next-tesla-retail-trader-on-2-year-anniversary-of-craze-150734011.html',
  'http

In [ ]:
#Search and scrape cleaned URLs
def scrape_and_process(URLs):
    ARTICLES = []
    for url in URLs: 
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = [paragraph.text for paragraph in paragraphs]
        words = ' '.join(text).split(' ')[:350]
        ARTICLE = ' '.join(words)
        ARTICLES.append(ARTICLE)
    return ARTICLES

In [ ]:
articles = {ticker:scrape_and_process(cleaned_urls[ticker]) for ticker in monitored_tickers}
articles

{'GME': ['This post has been updated. The end of an era is approaching for Bed Bath & Beyond (BBBY), once one of the U.S.\'s most beloved home furnishing retailers. Bed Bath & Beyond revealed in its latest regulatory filing that it\'s defaulted on its loans and lacks the financial means to repay its debt. The company also reiterated the possibility that it may seek bankruptcy protection in the near future. From simple beginnings to becoming a national retail titan that sold everything from coffee makers to candy, the company\'s more than 50-year long journey has been both adventurous and chaotic. Here\'s a look at the years leading up to its current near-death state. Founders Leonard Feinstein and Warren Eisenberg originally worked at Arlans, a discount retail chain, but eventually discovered the need for niche stores that better served shoppers. In 1971, they opened the first linens-only "Bed \'n Bath" store in Springfield, New Jersey. Their bet was spot-on: The 1980s brought a rise i

In [ ]:
articles['TSLA'][2]

'Shares of Meta Platforms surge after company results that topped revenue expectations If Cathie Wood had to hold one stock for the next decade, it would be Tesla, she said in an interview. Wood has been snapping up Tesla stock since shares of the EV maker closed out a dismal year in 2022. She estimates the stock will soar to $1,500 a share over the next five years, marking a 675% increase. If Cathie Wood had to buy one stock today and hold it for the next 10 years, it would be Tesla, the Ark Invest CEO said. In a recent Twitter Spaces interview with Cardone Capital, the famed money manager professed her bullishness for Tesla stock, which is the largest holding in her flagship ARK Innovation ETF. Though Tesla is still recovering from a dismal performance 2022, Wood has poured more money into shares of the electric-vehicle maker, snapping up $47 million of the stock in a single week in January. "Well, especially because of how much it\'s been pummeled recently, and we\'re known for this

In [ ]:
#Summarize all articles
def summarize(articles):
    summaries = []
    for article in articles:
        input_ids = tokenizer.encode(article, return_tensors='pt')
        output = model.generate(input_ids, max_length=55, num_beams=5, early_stopping=True)
        summary = tokenizer.decode(output[0], skip_special_tokens=True)
        summaries.append(summary)
    return summaries

In [ ]:
summaries = {ticker:summarize(articles[ticker]) for ticker in monitored_tickers}
summaries

{'GME': ['Company says it’s defaulted on its loans and lacks financial means. More than 50 years of growth, ups and downs',
  'It all started with a meteoric rise of video game retailer GME. The buying of shares became a self-fulfilling prophecy',
  'Top 12 one dollar stocks to buy among hedge funds. These are the most popular stocks among hedge funds.',
  'Retailer to hold conference call on Thursday. Shares have surged 105% in the past three days',
  'Video game retailer is set to report third-quarter results on Oct 26.',
  'Uerkwitz cuts price target on gaming retailer to $20 from $26.',
  'Beaten-down names including Tesla, Carvana, and Coinbase all advance. A mini meme trade was also in the works on Friday',
  'All images are copyrighted.',
  'Airline stocks, Tesla, AMC Entertainment lead the way. BlackRock, Bed Bath & Beyond among day’s gainers',
  'Two years ago, retail investors poured money into stocks. Activity related to meme stocks has tapered off',
  'Mobility industry evo

In [ ]:
summaries['BTC']

['CoinDesk Market Index (CMI) 1,101 18.9  1.7%',
 'Ether, Cardano lead gains among smaller coins. Bitcoin pares strong advance after hitting five-month high Wednesday',
 'Bitcoin drops below $22,600 on Monday, recording its biggest one-day percentage loss since November. Some $44 million worth of BTC long positions liquidated in past 24 hours',
 'Record 687 BTC produced in January 2023, 45% increase from prior month. Unrestricted Bitcoin Holdings increased to 8,090 BTC as of January 31',
 'Company mines record 697 bitcoins in January, up 50% from prior month. Mining fleet of 66,000 latest-generation miners up 214% from December 2022',
 'Paraguay farm with 2,888 new miners increasing hashrate 168 PH/s. Bitfarms to benefit from disciplined approach to investment as industry recovers',
 'Dogecoin is sole winner among top 10 cryptos on Tuesday afternoon. Asian stock markets drop as investors look ahead to U.S. policy announcements',
 'Altcoins’ gains have been part of post-Fed market upswi

Sentiment Analysis 

In [ ]:
from transformers import pipeline
sentiment = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
sentiment(summaries['BTC'])

[{'label': 'NEGATIVE', 'score': 0.9771423935890198},
 {'label': 'POSITIVE', 'score': 0.7815185785293579},
 {'label': 'NEGATIVE', 'score': 0.9996176958084106},
 {'label': 'POSITIVE', 'score': 0.9032465815544128},
 {'label': 'POSITIVE', 'score': 0.7537873387336731},
 {'label': 'POSITIVE', 'score': 0.7796147465705872},
 {'label': 'NEGATIVE', 'score': 0.9796889424324036},
 {'label': 'NEGATIVE', 'score': 0.9966051578521729},
 {'label': 'POSITIVE', 'score': 0.9608860015869141},
 {'label': 'POSITIVE', 'score': 0.9985652565956116}]

In [ ]:
scores = {ticker:sentiment(summaries[ticker]) for ticker in monitored_tickers}
scores

{'GME': [{'label': 'NEGATIVE', 'score': 0.9996053576469421},
  {'label': 'NEGATIVE', 'score': 0.9903987646102905},
  {'label': 'POSITIVE', 'score': 0.9996557235717773},
  {'label': 'POSITIVE', 'score': 0.9471070766448975},
  {'label': 'NEGATIVE', 'score': 0.9792588949203491},
  {'label': 'NEGATIVE', 'score': 0.9943915009498596},
  {'label': 'NEGATIVE', 'score': 0.9974543452262878},
  {'label': 'NEGATIVE', 'score': 0.9880996346473694},
  {'label': 'POSITIVE', 'score': 0.994852602481842},
  {'label': 'NEGATIVE', 'score': 0.9995341300964355},
  {'label': 'POSITIVE', 'score': 0.9941892623901367}],
 'TSLA': [{'label': 'NEGATIVE', 'score': 0.9633318781852722},
  {'label': 'NEGATIVE', 'score': 0.9676677584648132},
  {'label': 'NEGATIVE', 'score': 0.9900915622711182},
  {'label': 'NEGATIVE', 'score': 0.9974543452262878},
  {'label': 'NEGATIVE', 'score': 0.9977608919143677},
  {'label': 'NEGATIVE', 'score': 0.9798082709312439},
  {'label': 'POSITIVE', 'score': 0.9783797264099121},
  {'label': '

In [ ]:
print(summaries['GME'][3], scores['GME'][3]['label'], scores['GME'][3]['score'])

Retailer to hold conference call on Thursday. Shares have surged 105% in the past three days POSITIVE 0.9471070766448975


In [ ]:
scores['BTC'][0]['score']

0.9771423935890198

Exporting Results to CSV

In [ ]:
summaries

{'GME': ['Company says it’s defaulted on its loans and lacks financial means. More than 50 years of growth, ups and downs',
  'It all started with a meteoric rise of video game retailer GME. The buying of shares became a self-fulfilling prophecy',
  'Top 12 one dollar stocks to buy among hedge funds. These are the most popular stocks among hedge funds.',
  'Retailer to hold conference call on Thursday. Shares have surged 105% in the past three days',
  'Video game retailer is set to report third-quarter results on Oct 26.',
  'Uerkwitz cuts price target on gaming retailer to $20 from $26.',
  'Beaten-down names including Tesla, Carvana, and Coinbase all advance. A mini meme trade was also in the works on Friday',
  'All images are copyrighted.',
  'Airline stocks, Tesla, AMC Entertainment lead the way. BlackRock, Bed Bath & Beyond among day’s gainers',
  'Two years ago, retail investors poured money into stocks. Activity related to meme stocks has tapered off',
  'Mobility industry evo

In [ ]:
scores 

{'GME': [{'label': 'NEGATIVE', 'score': 0.9996053576469421},
  {'label': 'NEGATIVE', 'score': 0.9903987646102905},
  {'label': 'POSITIVE', 'score': 0.9996557235717773},
  {'label': 'POSITIVE', 'score': 0.9471070766448975},
  {'label': 'NEGATIVE', 'score': 0.9792588949203491},
  {'label': 'NEGATIVE', 'score': 0.9943915009498596},
  {'label': 'NEGATIVE', 'score': 0.9974543452262878},
  {'label': 'NEGATIVE', 'score': 0.9880996346473694},
  {'label': 'POSITIVE', 'score': 0.994852602481842},
  {'label': 'NEGATIVE', 'score': 0.9995341300964355},
  {'label': 'POSITIVE', 'score': 0.9941892623901367}],
 'TSLA': [{'label': 'NEGATIVE', 'score': 0.9633318781852722},
  {'label': 'NEGATIVE', 'score': 0.9676677584648132},
  {'label': 'NEGATIVE', 'score': 0.9900915622711182},
  {'label': 'NEGATIVE', 'score': 0.9974543452262878},
  {'label': 'NEGATIVE', 'score': 0.9977608919143677},
  {'label': 'NEGATIVE', 'score': 0.9798082709312439},
  {'label': 'POSITIVE', 'score': 0.9783797264099121},
  {'label': '

In [ ]:
cleaned_urls

{'GME': ['https://finance.yahoo.com/news/bed-bath-beyond-timeline-loan-default-bankruptcy-212526437.html',
  'https://finance.yahoo.com/news/meme-stock-mania-5-lessons-220010930.html',
  'https://finance.yahoo.com/news/12-best-one-dollar-stocks-143215032.html',
  'https://ca.finance.yahoo.com/news/bed-bath-beyond-surged-105-185829217.html',
  'https://finance.yahoo.com/news/strength-seen-gamestop-gme-14-144602277.html',
  'https://finance.yahoo.com/news/why-game-stop-stock-is-probably-dead-money-for-a-while-according-to-one-analyst-180851839.html',
  'https://finance.yahoo.com/news/tesla-arkk-meme-stocks-rally-2021-energy-technology-markets-195153458.html',
  'https://www.google.com/search?q%3Dyahoo%2Bfinance%2BGME%26tbm%3Dnws%26pccc%3D1',
  'https://finance.yahoo.com/news/midday-movers-airline-stocks-tesla-135534803.html',
  'https://finance.yahoo.com/news/meme-stocks-i-bought-amc-thinking-it-would-be-the-next-tesla-retail-trader-on-2-year-anniversary-of-craze-150734011.html',
  'http

In [ ]:
range(len(summaries['GME']))

range(0, 11)

In [ ]:
summaries['GME'][3]

'Retailer to hold conference call on Thursday. Shares have surged 105% in the past three days'

In [ ]:
def create_output_array(summaries, scores, urls):
    output = []
    for ticker in monitored_tickers:
        for counter in range(len(summaries[ticker])):
            output_this = [
                ticker,
                summaries[ticker][counter],
                scores[ticker][counter]['label'],
                scores[ticker][counter]['score'],
                urls[ticker][counter]
            ]
            output.append(output_this)
    return output

In [ ]:
final_output = create_output_array(summaries, scores, cleaned_urls)
final_output


[['GME',
  'Company says it’s defaulted on its loans and lacks financial means. More than 50 years of growth, ups and downs',
  'NEGATIVE',
  0.9996053576469421,
  'https://finance.yahoo.com/news/bed-bath-beyond-timeline-loan-default-bankruptcy-212526437.html'],
 ['GME',
  'It all started with a meteoric rise of video game retailer GME. The buying of shares became a self-fulfilling prophecy',
  'NEGATIVE',
  0.9903987646102905,
  'https://finance.yahoo.com/news/meme-stock-mania-5-lessons-220010930.html'],
 ['GME',
  'Top 12 one dollar stocks to buy among hedge funds. These are the most popular stocks among hedge funds.',
  'POSITIVE',
  0.9996557235717773,
  'https://finance.yahoo.com/news/12-best-one-dollar-stocks-143215032.html'],
 ['GME',
  'Retailer to hold conference call on Thursday. Shares have surged 105% in the past three days',
  'POSITIVE',
  0.9471070766448975,
  'https://ca.finance.yahoo.com/news/bed-bath-beyond-surged-105-185829217.html'],
 ['GME',
  'Video game retailer 

In [ ]:
final_output.insert(0, ['Ticker', 'Summary', 'Label', 'Confidence', 'URL'])

In [ ]:
final_output

[['Ticker', 'Summary', 'Label', 'Confidence', 'URL'],
 ['GME',
  'Company says it’s defaulted on its loans and lacks financial means. More than 50 years of growth, ups and downs',
  'NEGATIVE',
  0.9996053576469421,
  'https://finance.yahoo.com/news/bed-bath-beyond-timeline-loan-default-bankruptcy-212526437.html'],
 ['GME',
  'It all started with a meteoric rise of video game retailer GME. The buying of shares became a self-fulfilling prophecy',
  'NEGATIVE',
  0.9903987646102905,
  'https://finance.yahoo.com/news/meme-stock-mania-5-lessons-220010930.html'],
 ['GME',
  'Top 12 one dollar stocks to buy among hedge funds. These are the most popular stocks among hedge funds.',
  'POSITIVE',
  0.9996557235717773,
  'https://finance.yahoo.com/news/12-best-one-dollar-stocks-143215032.html'],
 ['GME',
  'Retailer to hold conference call on Thursday. Shares have surged 105% in the past three days',
  'POSITIVE',
  0.9471070766448975,
  'https://ca.finance.yahoo.com/news/bed-bath-beyond-surged-

In [ ]:
import csv
with open('assetsummaries.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerows(final_output)